# BLOQUE 1: Configuración Inicial y Detección Básica de Pose
### Instalación, imports y detección simple de landmarks corporales

Detección básica de pose con MediaPipe - Versión mínima funcional

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# ==================== INICIALIZACIÓN MEDIAPIPE ====================
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Configuración básica de pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

print(" MediaPipe inicializado correctamente")

# ==================== FUNCIÓN DE VISUALIZACIÓN SIMPLE ====================
def procesar_video_basico():
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print(" Error: No se puede acceder a la cámara")
        return

    print(" Presiona 'q' para salir")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Voltear horizontalmente para efecto espejo
        frame = cv2.flip(frame, 1)

        # Convertir BGR a RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Procesar con MediaPipe
        results = pose.process(rgb_frame)

        # Dibujar landmarks si se detectan
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )

            # Mostrar información básica
            cv2.putText(frame, "Pose detectada", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "Sin deteccion", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Mostrar frame
        cv2.imshow('BLOQUE 1 - Deteccion Basica', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    pose.close()
    print(" Sistema cerrado")

# Ejecutar
procesar_video_basico()


 MediaPipe inicializado correctamente
 Presiona 'q' para salir
Sistema cerrado


# BLOQUE 2: Análisis de Distancia y Detección Automática de Planos
### Calcula distancias corporales y clasifica planos cinematográficos automáticamente

Sistema que analiza la distancia del sujeto y clasifica planos cinematográficos

In [5]:

import cv2
import mediapipe as mp
import numpy as np
from collections import deque

# Inicialización
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# ==================== DEFINICIÓN DE PLANOS CINEMATOGRÁFICOS ====================
PLANOS_BASICOS = {
    'EXTREME_WIDE': {'nombre': 'Plano General Extremo', 'descripcion': 'Persona muy lejos'},
    'WIDE': {'nombre': 'Plano General', 'descripcion': 'Cuerpo completo + espacio'},
    'FULL': {'nombre': 'Plano Entero', 'descripcion': 'De pies a cabeza'},
    'COWBOY': {'nombre': 'Plano Americano', 'descripcion': 'Desde rodillas'},
    'MEDIUM': {'nombre': 'Plano Medio', 'descripcion': 'Desde cintura'},
    'CLOSEUP': {'nombre': 'Primer Plano', 'descripcion': 'Cara y hombros'},
    'EXTREME_CLOSEUP': {'nombre': 'Primerisimo Plano', 'descripcion': 'Solo rostro'}
}

# ==================== DETECTOR DE PLANOS ====================
class DetectorPlanos:
    def __init__(self):
        self.historial = deque(maxlen=5)  # Suavizado

    def detectar_plano(self, pose_landmarks):
        if not pose_landmarks:
            return 'MEDIUM'

        lm = pose_landmarks.landmark

        # Calcular ancho de hombros (métrica principal)
        ancho_hombros = abs(lm[11].x - lm[12].x)

        # Visibilidad de partes del cuerpo
        hips_visible = min(lm[23].visibility, lm[24].visibility)
        knees_visible = min(lm[25].visibility, lm[26].visibility)
        ankles_visible = min(lm[27].visibility, lm[28].visibility)

        # Clasificación por ancho de hombros
        if ancho_hombros > 0.50:
            plano = 'EXTREME_CLOSEUP'
        elif ancho_hombros > 0.38:
            plano = 'CLOSEUP'
        elif ancho_hombros > 0.28:
            plano = 'MEDIUM'
        elif ancho_hombros > 0.20:
            if knees_visible > 0.3:
                plano = 'COWBOY'
            else:
                plano = 'MEDIUM'
        elif ancho_hombros > 0.15:
            if ankles_visible > 0.3:
                plano = 'FULL'
            else:
                plano = 'COWBOY'
        elif ancho_hombros > 0.10:
            plano = 'WIDE'
        else:
            plano = 'EXTREME_WIDE'

        # Suavizado
        self.historial.append(plano)
        from collections import Counter
        plano_final = Counter(self.historial).most_common(1)[0][0]

        return plano_final

# ==================== FUNCIÓN PRINCIPAL ====================
def procesar_con_deteccion_planos():
    cap = cv2.VideoCapture(0)
    detector = DetectorPlanos()

    print(" BLOQUE 2: Detección automática de planos")
    print("Aléjate o acércate a la cámara para ver diferentes planos")
    print("Presiona 'q' para salir\n")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        h, w = frame.shape[:2]

        # Procesar
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)

        # Detectar plano
        plano_actual = 'MEDIUM'
        ancho_hombros = 0

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )

            plano_actual = detector.detectar_plano(results.pose_landmarks)

            # Calcular ancho para debug
            lm = results.pose_landmarks.landmark
            ancho_hombros = abs(lm[11].x - lm[12].x)

        # ==================== INTERFAZ ====================
        # Panel superior
        cv2.rectangle(frame, (0, 0), (w, 120), (0, 0, 0), -1)
        cv2.rectangle(frame, (0, 0), (w, 120), (0, 255, 255), 2)

        # Información del plano
        info_plano = PLANOS_BASICOS[plano_actual]
        cv2.putText(frame, f"PLANO: {info_plano['nombre']}", (10, 35),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(frame, info_plano['descripcion'], (10, 65),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 1)

        # Debug
        cv2.putText(frame, f"Ancho hombros: {ancho_hombros:.3f}", (10, 95),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150, 150, 150), 1)

        # Grid de tercios
        color_grid = (80, 80, 80)
        cv2.line(frame, (w//3, 0), (w//3, h), color_grid, 1)
        cv2.line(frame, (2*w//3, 0), (2*w//3, h), color_grid, 1)
        cv2.line(frame, (0, h//3), (w, h//3), color_grid, 1)
        cv2.line(frame, (0, 2*h//3), (w, 2*h//3), color_grid, 1)

        cv2.imshow('BLOQUE 2 - Deteccion Automatica', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    pose.close()
    print(" Sistema cerrado")

# Ejecutar
procesar_con_deteccion_planos()


 BLOQUE 2: Detección automática de planos
Aléjate o acércate a la cámara para ver diferentes planos
Presiona 'q' para salir

 Sistema cerrado


# BLOQUE 3: Control Manual con Gestos de Mano
### Añade detección de manos y gestos para control manual de planos

Sistema con detección de manos para cambiar planos manualmente

In [6]:
import cv2
import mediapipe as mp
import numpy as np
from collections import deque

# Inicialización
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.6
)

# ==================== PLANOS CON GESTOS ====================
PLANOS_CON_GESTOS = {
    'EXTREME_WIDE': {'nombre': 'General Extremo', 'gesto': '👎 Pulgar abajo'},
    'WIDE': {'nombre': 'Plano General', 'gesto': '✊ Puño cerrado'},
    'FULL': {'nombre': 'Plano Entero', 'gesto': '☝️ 1 dedo'},
    'COWBOY': {'nombre': 'Plano Americano', 'gesto': '🖖 4 dedos'},
    'MEDIUM': {'nombre': 'Plano Medio', 'gesto': '🤟 3 dedos'},
    'MEDIUM_CLOSEUP': {'nombre': 'Medio Corto', 'gesto': '🖐️ Mano abierta'},
    'CLOSEUP': {'nombre': 'Primer Plano', 'gesto': '✌️ Paz (V)'},
    'EXTREME_CLOSEUP': {'nombre': 'Primerisimo', 'gesto': '🤘 Rock'}
}

# ==================== DETECTOR DE GESTOS ====================
def contar_dedos(hand_landmarks):
    dedos = 0
    tips_ids = [4, 8, 12, 16, 20]

    # Pulgar
    if hand_landmarks.landmark[tips_ids[0]].x < hand_landmarks.landmark[tips_ids[0] - 1].x:
        dedos += 1

    # Otros dedos
    for i in range(1, 5):
        if hand_landmarks.landmark[tips_ids[i]].y < hand_landmarks.landmark[tips_ids[i] - 2].y:
            dedos += 1

    return dedos

def detectar_rock(hand_landmarks):
    indice_up = hand_landmarks.landmark[8].y < hand_landmarks.landmark[6].y
    medio_down = hand_landmarks.landmark[12].y > hand_landmarks.landmark[10].y
    anular_down = hand_landmarks.landmark[16].y > hand_landmarks.landmark[14].y
    meñique_up = hand_landmarks.landmark[20].y < hand_landmarks.landmark[18].y
    return indice_up and medio_down and anular_down and meñique_up

def detectar_pulgar_abajo(hand_landmarks):
    pulgar_down = hand_landmarks.landmark[4].y > hand_landmarks.landmark[3].y
    otros_cerrados = all([
        hand_landmarks.landmark[8].y > hand_landmarks.landmark[6].y,
        hand_landmarks.landmark[12].y > hand_landmarks.landmark[10].y,
        hand_landmarks.landmark[16].y > hand_landmarks.landmark[14].y,
        hand_landmarks.landmark[20].y > hand_landmarks.landmark[18].y
    ])
    return pulgar_down and otros_cerrados

def detectar_paz(hand_landmarks):
    indice_up = hand_landmarks.landmark[8].y < hand_landmarks.landmark[6].y
    medio_up = hand_landmarks.landmark[12].y < hand_landmarks.landmark[10].y
    anular_down = hand_landmarks.landmark[16].y > hand_landmarks.landmark[14].y
    meñique_down = hand_landmarks.landmark[20].y > hand_landmarks.landmark[18].y
    return indice_up and medio_up and anular_down and meñique_down

def clasificar_gesto(hand_landmarks):
    dedos = contar_dedos(hand_landmarks)

    if detectar_rock(hand_landmarks):
        return 'EXTREME_CLOSEUP', "🤘"
    elif detectar_pulgar_abajo(hand_landmarks):
        return 'EXTREME_WIDE', "👎"
    elif dedos == 0:
        return 'WIDE', "✊"
    elif detectar_paz(hand_landmarks):
        return 'CLOSEUP', "✌️"
    elif dedos == 1:
        return 'FULL', "☝️"
    elif dedos == 3:
        return 'MEDIUM', "🤟"
    elif dedos == 4:
        return 'COWBOY', "🖖"
    elif dedos == 5:
        return 'MEDIUM_CLOSEUP', "🖐️"
    else:
        return 'MEDIUM', str(dedos)

# ==================== FUNCIÓN PRINCIPAL ====================
def procesar_con_gestos():
    cap = cv2.VideoCapture(0)
    plano_actual = 'MEDIUM'
    ultimo_gesto = ""

    print(" BLOQUE 3: Control manual con gestos")
    print("Usa gestos de mano para cambiar planos")
    print("Presiona 'q' para salir\n")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        h, w = frame.shape[:2]

        # Procesar
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pose_results = pose.process(rgb_frame)
        hands_results = hands.process(rgb_frame)

        # Dibujar pose
        if pose_results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )

        # Detectar gestos
        if hands_results.multi_hand_landmarks:
            for hand_landmarks in hands_results.multi_hand_landmarks:
                # Dibujar mano
                mp_drawing.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )

                # Clasificar gesto
                plano_actual, ultimo_gesto = clasificar_gesto(hand_landmarks)

        # ==================== INTERFAZ ====================
        # Panel superior
        cv2.rectangle(frame, (0, 0), (w, 150), (0, 0, 0), -1)
        cv2.rectangle(frame, (0, 0), (w, 150), (255, 100, 255), 2)

        cv2.putText(frame, "MODO: MANUAL", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 100, 255), 2)

        info = PLANOS_CON_GESTOS[plano_actual]
        cv2.putText(frame, f"PLANO: {info['nombre']}", (10, 65),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"Gesto: {info['gesto']}", (10, 95),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 1)

        if ultimo_gesto:
            cv2.putText(frame, f"Detectado: {ultimo_gesto}", (10, 125),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Panel de ayuda lateral
        panel_x = w - 250
        cv2.rectangle(frame, (panel_x, 0), (w, 350), (0, 0, 0), -1)
        cv2.rectangle(frame, (panel_x, 0), (w, 350), (255, 100, 255), 2)

        cv2.putText(frame, "GESTOS:", (panel_x + 10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        y = 60
        for key, plano in list(PLANOS_CON_GESTOS.items())[:8]:
            cv2.putText(frame, plano['gesto'][:8], (panel_x + 10, y),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, (200, 200, 200), 1)
            y += 35

        cv2.imshow('BLOQUE 3 - Control Manual', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    pose.close()
    hands.close()
    print(" Sistema cerrado")

# Ejecutar
procesar_con_gestos()


 BLOQUE 3: Control manual con gestos
Usa gestos de mano para cambiar planos
Presiona 'q' para salir

 Sistema cerrado


# BLOQUE 4: Encuadre Inteligente y Seguimiento Suave
### Sistema de zoom y centrado automático con transiciones cinematográficas

Sistema con zoom, seguimiento del sujeto y transiciones suaves

In [7]:
import cv2
import mediapipe as mp
import numpy as np
from collections import deque
from dataclasses import dataclass

# Inicialización
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.7
)

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.6
)

# ==================== PLANOS CON ZOOM ====================
PLANOS_ZOOM = {
    'EXTREME_WIDE': {'nombre': 'General Extremo', 'zoom': 0.7, 'y_offset': 0.05},
    'WIDE': {'nombre': 'Plano General', 'zoom': 0.9, 'y_offset': 0.0},
    'FULL': {'nombre': 'Plano Entero', 'zoom': 1.0, 'y_offset': 0.0},
    'COWBOY': {'nombre': 'Americano', 'zoom': 1.3, 'y_offset': -0.05},
    'MEDIUM': {'nombre': 'Plano Medio', 'zoom': 1.5, 'y_offset': -0.08},
    'MEDIUM_CLOSEUP': {'nombre': 'Medio Corto', 'zoom': 1.8, 'y_offset': -0.10},
    'CLOSEUP': {'nombre': 'Primer Plano', 'zoom': 2.2, 'y_offset': -0.08},
    'EXTREME_CLOSEUP': {'nombre': 'Primerisimo', 'zoom': 2.8, 'y_offset': -0.05}
}

# ==================== SISTEMA DE ENCUADRE ====================
@dataclass
class FrameTarget:
    x: float = 0.5
    y: float = 0.5
    zoom: float = 1.0

class SmoothFramer:
    def __init__(self, smoothing=0.15):
        self.current = FrameTarget()
        self.target = FrameTarget()
        self.smoothing = smoothing

    def update(self, center_x, center_y, zoom, y_offset=0.0):
        self.target.x = center_x
        self.target.y = center_y + y_offset
        self.target.zoom = zoom

        # Interpolación suave
        self.current.x += (self.target.x - self.current.x) * self.smoothing
        self.current.y += (self.target.y - self.current.y) * self.smoothing
        self.current.zoom += (self.target.zoom - self.current.zoom) * self.smoothing

        return self.current

def obtener_centro_sujeto(pose_landmarks, plano_actual):
    if not pose_landmarks:
        return (0.5, 0.5)

    lm = pose_landmarks.landmark

    if plano_actual in ['CLOSEUP', 'EXTREME_CLOSEUP']:
        # Centrar en rostro
        nose = np.array([lm[0].x, lm[0].y])
        mouth_left = np.array([lm[9].x, lm[9].y])
        mouth_right = np.array([lm[10].x, lm[10].y])
        mouth_center = (mouth_left + mouth_right) / 2
        face_center = (nose + mouth_center) / 2
        return (float(face_center[0]), float(face_center[1]))

    elif plano_actual in ['MEDIUM', 'MEDIUM_CLOSEUP']:
        # Centrar en torso superior
        nose = np.array([lm[0].x, lm[0].y])
        shoulders = (np.array([lm[11].x, lm[11].y]) + np.array([lm[12].x, lm[12].y])) / 2
        center = (nose + shoulders * 2) / 3
        return (float(center[0]), float(center[1]))

    else:
        # Centrar en cuerpo completo
        key_points = [lm[0], lm[11], lm[12], lm[23], lm[24]]
        center_x = np.mean([p.x for p in key_points])
        center_y = np.mean([p.y for p in key_points])
        return (float(center_x), float(center_y))

def aplicar_encuadre(frame, framer_state):
    h, w = frame.shape[:2]

    zoom = framer_state.zoom
    crop_w = int(w / zoom)
    crop_h = int(h / zoom)

    center_x = int(framer_state.x * w)
    center_y = int(framer_state.y * h)

    x1 = max(0, center_x - crop_w // 2)
    y1 = max(0, center_y - crop_h // 2)
    x2 = min(w, x1 + crop_w)
    y2 = min(h, y1 + crop_h)

    if x2 - x1 < crop_w:
        x1 = max(0, x2 - crop_w)
    if y2 - y1 < crop_h:
        y1 = max(0, y2 - crop_h)

    cropped = frame[y1:y2, x1:x2]
    if cropped.size == 0:
        return frame

    resized = cv2.resize(cropped, (w, h), interpolation=cv2.INTER_LINEAR)
    return resized

# ==================== DETECCIÓN DE GESTOS (simplificado) ====================
def detectar_gesto_simple(hand_landmarks):
    dedos = 0
    tips_ids = [4, 8, 12, 16, 20]

    if hand_landmarks.landmark[tips_ids[0]].x < hand_landmarks.landmark[tips_ids[0] - 1].x:
        dedos += 1
    for i in range(1, 5):
        if hand_landmarks.landmark[tips_ids[i]].y < hand_landmarks.landmark[tips_ids[i] - 2].y:
            dedos += 1

    if dedos == 0:
        return 'WIDE'
    elif dedos == 1:
        return 'FULL'
    elif dedos == 2:
        return 'CLOSEUP'
    elif dedos == 3:
        return 'MEDIUM'
    elif dedos == 5:
        return 'MEDIUM_CLOSEUP'
    else:
        return 'MEDIUM'

# ==================== FUNCIÓN PRINCIPAL ====================
def procesar_con_encuadre():
    cap = cv2.VideoCapture(0)
    framer = SmoothFramer(smoothing=0.15)
    plano_actual = 'MEDIUM'

    print(" BLOQUE 4: Encuadre inteligente con zoom")
    print("El sistema seguirá y centrará automáticamente al sujeto")
    print("Usa gestos de mano para cambiar planos")
    print("Presiona 'q' para salir\n")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        h, w = frame.shape[:2]

        # Procesar
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pose_results = pose.process(rgb_frame)
        hands_results = hands.process(rgb_frame)

        # Frame de detección (con landmarks)
        frame_deteccion = frame.copy()

        if pose_results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame_deteccion, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )

        if hands_results.multi_hand_landmarks:
            for hand_landmarks in hands_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame_deteccion, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )
                plano_actual = detectar_gesto_simple(hand_landmarks)

        # Calcular centro y aplicar encuadre
        centro = obtener_centro_sujeto(pose_results.pose_landmarks, plano_actual)
        zoom_factor = PLANOS_ZOOM[plano_actual]['zoom']
        y_offset = PLANOS_ZOOM[plano_actual]['y_offset']

        framer_state = framer.update(centro[0], centro[1], zoom_factor, y_offset)

        # Frame resultado (con encuadre aplicado)
        frame_resultado = aplicar_encuadre(frame.copy(), framer_state)

        # Grid en resultado
        color_grid = (80, 80, 80)
        cv2.line(frame_resultado, (w//3, 0), (w//3, h), color_grid, 1)
        cv2.line(frame_resultado, (2*w//3, 0), (2*w//3, h), color_grid, 1)
        cv2.line(frame_resultado, (0, h//3), (w, h//3), color_grid, 1)
        cv2.line(frame_resultado, (0, 2*h//3), (w, 2*h//3), color_grid, 1)

        # Info en resultado
        cv2.putText(frame_resultado, f"PLANO: {PLANOS_ZOOM[plano_actual]['nombre']}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(frame_resultado, f"Zoom: {zoom_factor:.1f}x", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 1)

        # Info en detección
        cv2.putText(frame_deteccion, "DETECCION (landmarks)", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

        # Mostrar ambas ventanas
        cv2.imshow('DETECCION - Landmarks', frame_deteccion)
        cv2.imshow('RESULTADO - Encuadre Final', frame_resultado)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    pose.close()
    hands.close()
    print(" Sistema cerrado")

# Ejecutar
procesar_con_encuadre()


 BLOQUE 4: Encuadre inteligente con zoom
El sistema seguirá y centrará automáticamente al sujeto
Usa gestos de mano para cambiar planos
Presiona 'q' para salir

 Sistema cerrado


# BLOQUE 5: Sistema Híbrido AUTO/MANUAL con Interfaz Completa
### Integración completa: detección automática + control manual + panel de estado

Combina detección automática, control manual y panel de información

In [8]:
import cv2
import mediapipe as mp
import numpy as np
from collections import deque, Counter
from dataclasses import dataclass
import time

# Inicialización
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.7
)

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.6
)

# ==================== PLANOS COMPLETOS ====================
PLANOS = {
    'EXTREME_WIDE': {'nombre': 'General Extremo', 'zoom': 0.7, 'y_offset': 0.05},
    'WIDE': {'nombre': 'Plano General', 'zoom': 0.9, 'y_offset': 0.0},
    'FULL': {'nombre': 'Plano Entero', 'zoom': 1.0, 'y_offset': 0.0},
    'COWBOY': {'nombre': 'Americano', 'zoom': 1.3, 'y_offset': -0.05},
    'MEDIUM': {'nombre': 'Plano Medio', 'zoom': 1.5, 'y_offset': -0.08},
    'MEDIUM_CLOSEUP': {'nombre': 'Medio Corto', 'zoom': 1.8, 'y_offset': -0.10},
    'CLOSEUP': {'nombre': 'Primer Plano', 'zoom': 2.2, 'y_offset': -0.08},
    'EXTREME_CLOSEUP': {'nombre': 'Primerisimo', 'zoom': 2.8, 'y_offset': -0.05}
}

# ==================== DETECTOR AUTOMÁTICO ====================
class DetectorAutomatico:
    def __init__(self):
        self.historial = deque(maxlen=5)

    def detectar(self, pose_landmarks):
        if not pose_landmarks:
            return 'MEDIUM'

        lm = pose_landmarks.landmark
        ancho_hombros = abs(lm[11].x - lm[12].x)
        knees_vis = min(lm[25].visibility, lm[26].visibility)
        ankles_vis = min(lm[27].visibility, lm[28].visibility)

        if ancho_hombros > 0.50:
            plano = 'EXTREME_CLOSEUP'
        elif ancho_hombros > 0.38:
            plano = 'CLOSEUP'
        elif ancho_hombros > 0.28:
            plano = 'MEDIUM'
        elif ancho_hombros > 0.20:
            plano = 'COWBOY' if knees_vis > 0.3 else 'MEDIUM'
        elif ancho_hombros > 0.15:
            plano = 'FULL' if ankles_vis > 0.3 else 'COWBOY'
        elif ancho_hombros > 0.10:
            plano = 'WIDE'
        else:
            plano = 'EXTREME_WIDE'

        self.historial.append(plano)
        return Counter(self.historial).most_common(1)[0][0]

# ==================== DETECTOR DE GESTOS ====================
def clasificar_gesto(hand_landmarks):
    dedos = 0
    tips_ids = [4, 8, 12, 16, 20]

    if hand_landmarks.landmark[tips_ids[0]].x < hand_landmarks.landmark[tips_ids[0] - 1].x:
        dedos += 1
    for i in range(1, 5):
        if hand_landmarks.landmark[tips_ids[i]].y < hand_landmarks.landmark[tips_ids[i] - 2].y:
            dedos += 1

    mapeo = {0: 'WIDE', 1: 'FULL', 2: 'CLOSEUP', 3: 'MEDIUM', 
             4: 'COWBOY', 5: 'MEDIUM_CLOSEUP'}
    return mapeo.get(dedos, 'MEDIUM')

# ==================== SISTEMA DE ENCUADRE ====================
@dataclass
class FrameTarget:
    x: float = 0.5
    y: float = 0.5
    zoom: float = 1.0

class SmoothFramer:
    def __init__(self, smoothing=0.15):
        self.current = FrameTarget()
        self.target = FrameTarget()
        self.smoothing = smoothing

    def update(self, center_x, center_y, zoom, y_offset=0.0):
        self.target.x = center_x
        self.target.y = center_y + y_offset
        self.target.zoom = zoom

        self.current.x += (self.target.x - self.current.x) * self.smoothing
        self.current.y += (self.target.y - self.current.y) * self.smoothing
        self.current.zoom += (self.target.zoom - self.current.zoom) * self.smoothing

        return self.current

def obtener_centro(pose_landmarks, plano):
    if not pose_landmarks:
        return (0.5, 0.5)

    lm = pose_landmarks.landmark

    if plano in ['CLOSEUP', 'EXTREME_CLOSEUP']:
        nose = np.array([lm[0].x, lm[0].y])
        mouth = (np.array([lm[9].x, lm[9].y]) + np.array([lm[10].x, lm[10].y])) / 2
        center = (nose + mouth) / 2
        return (float(center[0]), float(center[1]))
    else:
        pts = [lm[0], lm[11], lm[12], lm[23], lm[24]]
        cx = np.mean([p.x for p in pts])
        cy = np.mean([p.y for p in pts])
        return (float(cx), float(cy))

def aplicar_encuadre(frame, framer_state):
    h, w = frame.shape[:2]
    zoom = framer_state.zoom
    crop_w = int(w / zoom)
    crop_h = int(h / zoom)

    center_x = int(framer_state.x * w)
    center_y = int(framer_state.y * h)

    x1 = max(0, center_x - crop_w // 2)
    y1 = max(0, center_y - crop_h // 2)
    x2 = min(w, x1 + crop_w)
    y2 = min(h, y1 + crop_h)

    if x2 - x1 < crop_w:
        x1 = max(0, x2 - crop_w)
    if y2 - y1 < crop_h:
        y1 = max(0, y2 - crop_h)

    cropped = frame[y1:y2, x1:x2]
    if cropped.size == 0:
        return frame

    return cv2.resize(cropped, (w, h), interpolation=cv2.INTER_LINEAR)

# ==================== PANEL DE CONTROL ====================
def crear_panel(modo, plano_actual, plano_auto, fps, show_grid):
    panel = np.zeros((720, 350, 3), dtype=np.uint8)
    y = 30

    cv2.putText(panel, "SISTEMA CAMARA", (10, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
    y += 50

    cv2.putText(panel, f"FPS: {fps:.1f}", (10, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)
    y += 40

    cv2.line(panel, (10, y), (340, y), (50, 50, 50), 1)
    y += 30

    color_modo = (0, 255, 255) if modo == 'AUTO' else (255, 100, 255)
    cv2.putText(panel, f"MODO: {modo}", (10, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color_modo, 2)
    y += 40

    cv2.putText(panel, "PLANO ACTIVO:", (10, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (100, 200, 255), 1)
    y += 25
    cv2.putText(panel, PLANOS[plano_actual]['nombre'], (10, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    y += 35

    if modo == 'AUTO' and plano_auto != plano_actual:
        cv2.putText(panel, f"Detectado: {PLANOS[plano_auto]['nombre']}", (10, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (150, 150, 255), 1)
        y += 25

    y += 10
    cv2.line(panel, (10, y), (340, y), (50, 50, 50), 1)
    y += 30

    cv2.putText(panel, "CONTROLES:", (10, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (100, 200, 255), 1)
    y += 30

    controles = [
        "'m' - Cambiar AUTO/MANUAL",
        "'g' - Grid ON/OFF",
        "'r' - Reset sistema",
        "Gestos mano (MANUAL):",
        "  0 dedos - General",
        "  1 dedo - Entero",
        "  2 dedos - Primer plano",
        "  3 dedos - Medio",
        "  5 dedos - Medio corto",
        "'q' - Salir"
    ]

    for ctrl in controles:
        cv2.putText(panel, ctrl, (10, y),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.45, (200, 200, 200), 1)
        y += 25

    y += 10
    cv2.line(panel, (10, y), (340, y), (50, 50, 50), 1)
    y += 30

    cv2.putText(panel, "ESTADO:", (10, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (100, 200, 255), 1)
    y += 30

    grid_color = (0, 255, 0) if show_grid else (100, 100, 100)
    cv2.putText(panel, f"Grid: {'ON' if show_grid else 'OFF'}", (10, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, grid_color, 1)

    return panel

# ==================== FUNCIÓN PRINCIPAL ====================
def sistema_completo():
    cap = cv2.VideoCapture(0)
    detector_auto = DetectorAutomatico()
    framer = SmoothFramer(smoothing=0.15)
    fps_history = deque(maxlen=30)

    modo = 'AUTO'
    plano_actual = 'MEDIUM'
    plano_auto = 'MEDIUM'
    show_grid = True

    print("=" * 60)
    print(" BLOQUE 5: SISTEMA HÍBRIDO COMPLETO")
    print("=" * 60)
    print("\n CARACTERÍSTICAS:")
    print("  • Detección automática de planos por distancia")
    print("  • Control manual con gestos de mano")
    print("  • Encuadre inteligente con seguimiento")
    print("  • Transiciones suaves cinematográficas")
    print("  • Panel de información en tiempo real")
    print("\n  'm' = Cambiar modo | 'g' = Grid | 'r' = Reset | 'q' = Salir\n")

    cv2.namedWindow('PANEL CONTROL', cv2.WINDOW_NORMAL)
    cv2.namedWindow('DETECCION', cv2.WINDOW_NORMAL)
    cv2.namedWindow('RESULTADO FINAL', cv2.WINDOW_NORMAL)

    cv2.resizeWindow('PANEL CONTROL', 350, 720)
    cv2.resizeWindow('DETECCION', 640, 360)
    cv2.resizeWindow('RESULTADO FINAL', 640, 360)

    while True:
        start_time = time.time()

        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        h, w = frame.shape[:2]

        # Procesar
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pose_results = pose.process(rgb_frame)
        hands_results = hands.process(rgb_frame)

        # Frame detección
        frame_deteccion = frame.copy()
        if pose_results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame_deteccion, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )

        if hands_results.multi_hand_landmarks:
            for hand_landmarks in hands_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame_deteccion, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )

        cv2.putText(frame_deteccion, "DETECCION + LANDMARKS", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

        # Detección automática
        plano_auto = detector_auto.detectar(pose_results.pose_landmarks)

        # Control manual
        if modo == 'MANUAL' and hands_results.multi_hand_landmarks:
            for hand_landmarks in hands_results.multi_hand_landmarks:
                plano_actual = clasificar_gesto(hand_landmarks)
        elif modo == 'AUTO':
            plano_actual = plano_auto

        # Encuadre
        centro = obtener_centro(pose_results.pose_landmarks, plano_actual)
        zoom_factor = PLANOS[plano_actual]['zoom']
        y_offset = PLANOS[plano_actual]['y_offset']

        framer_state = framer.update(centro[0], centro[1], zoom_factor, y_offset)
        frame_resultado = aplicar_encuadre(frame.copy(), framer_state)

        # Grid
        if show_grid:
            color_grid = (80, 80, 80)
            cv2.line(frame_resultado, (w//3, 0), (w//3, h), color_grid, 1)
            cv2.line(frame_resultado, (2*w//3, 0), (2*w//3, h), color_grid, 1)
            cv2.line(frame_resultado, (0, h//3), (w, h//3), color_grid, 1)
            cv2.line(frame_resultado, (0, 2*h//3), (w, 2*h//3), color_grid, 1)

        # Info resultado
        cv2.putText(frame_resultado, f"PLANO: {PLANOS[plano_actual]['nombre']}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame_resultado, f"Zoom: {zoom_factor:.1f}x | Modo: {modo}", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)

        # FPS
        fps = 1.0 / (time.time() - start_time)
        fps_history.append(fps)
        avg_fps = np.mean(fps_history)

        # Panel
        panel = crear_panel(modo, plano_actual, plano_auto, avg_fps, show_grid)

        # Mostrar
        cv2.imshow('PANEL CONTROL', panel)
        cv2.imshow('DETECCION', frame_deteccion)
        cv2.imshow('RESULTADO FINAL', frame_resultado)

        # Controles
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            print("\n Cerrando sistema...")
            break
        elif key == ord('m'):
            modo = 'MANUAL' if modo == 'AUTO' else 'AUTO'
            print(f" Modo cambiado a: {modo}")
        elif key == ord('g'):
            show_grid = not show_grid
            print(f" Grid: {'ON' if show_grid else 'OFF'}")
        elif key == ord('r'):
            framer = SmoothFramer(smoothing=0.15)
            detector_auto = DetectorAutomatico()
            plano_actual = 'MEDIUM'
            print(" Sistema reseteado")

    cap.release()
    cv2.destroyAllWindows()
    pose.close()
    hands.close()
    print(" Sistema cerrado correctamente")

# Ejecutar
sistema_completo()


 BLOQUE 5: SISTEMA HÍBRIDO COMPLETO

 CARACTERÍSTICAS:
  • Detección automática de planos por distancia
  • Control manual con gestos de mano
  • Encuadre inteligente con seguimiento
  • Transiciones suaves cinematográficas
  • Panel de información en tiempo real

  'm' = Cambiar modo | 'g' = Grid | 'r' = Reset | 'q' = Salir

 Modo cambiado a: MANUAL

 Cerrando sistema...
 Sistema cerrado correctamente
